In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import contractions
from sklearn.model_selection import StratifiedShuffleSplit


In [11]:
from time import time
import numpy as np
import pandas as pd
import tensorflow as tf

# Import Regex to clean up tweets
import re

import nltk, string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Get Reviews
import requests
import json

# Get Tweets
import httplib2
import requests
import urllib3

# TF IDF Imports


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from joblib import dump, load

# RNN & LSTM Imports

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.layers import SimpleRNN

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
df = pd.read_csv('./datasets/filtered_data.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww matches background colour seemingly stuck...,0,0,0,0,0,0
2,000113f07ec002fd,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0


In [13]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df.comment_text.values, df.toxic.values, 
                                                  stratify=df.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [14]:
token = Tokenizer(num_words=5000)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain) # transforme chaque texte de l'ensemble d'entrainement en une séquence d'entiers
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len) # Les séquences d'entiers sont complétées avec des zéros à la fin pour qu'elles aient toutes la même longueur maximale
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [15]:
%%time
strategy = tf.distribute.get_strategy()
with strategy.scope():
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 300)         14697600  
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               40100     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 14737801 (56.22 MB)
Trainable params: 14737801 (56.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
CPU times: user 79.8 ms, sys: 0 ns, total: 79.8 ms
Wall time: 115 ms


In [16]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1.0)
config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
session = tf.compat.v1.Session(config=config)

2024-03-13 11:38:26.241125: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 11:38:26.241319: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 11:38:26.241371: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 11:38:26.241640: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-13 11:38:26.241693: I tensorflow/core/common_runtime/gpu/gpu

In [17]:
history = model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
150/150 [==============================] - 290s 2s/step - loss: 0.2910 - accuracy: 0.9066 - val_loss: 0.2547 - val_accuracy: 0.9225
Epoch 2/5
150/150 [==============================] - 267s 2s/step - loss: 0.1256 - accuracy: 0.9583 - val_loss: 0.1617 - val_accuracy: 0.9433
Epoch 3/5
150/150 [==============================] - 259s 2s/step - loss: 0.0422 - accuracy: 0.9863 - val_loss: 0.1905 - val_accuracy: 0.9391
Epoch 4/5
150/150 [==============================] - 277s 2s/step - loss: 0.0313 - accuracy: 0.9910 - val_loss: 0.2304 - val_accuracy: 0.9266
Epoch 5/5
150/150 [==============================] - 271s 2s/step - loss: 0.0136 - accuracy: 0.9964 - val_loss: 0.2288 - val_accuracy: 0.9396


In [18]:

# affichage de l'accuracy
training_accuracy = history.history['accuracy'][-1]
print("Training Accuracy:", training_accuracy)

Training Accuracy: 0.9963518977165222
